In [1]:
#import nltk
#nltk.download('stopwords')

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
#import plotly.plotly as py
#import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
#cufflinks.go_offline()
#cufflinks.set_config_file(world_readable=True, theme='pearl')

Using TensorFlow backend.


In [3]:
df1 = pd.read_csv('data_sgd/train.tsv', header=None, sep="\t")

In [4]:
df1.head()

,0,1
0,__label__1,"sounds good, i want to buy tickets."
1,__label__3,please search a hotel for me there
2,__label__4,nothing. just relax.
3,__label__5,shall i book a table here?
4,__label__6,"no, not now.i need to check my account balance?"


In [5]:
df2 = pd.read_csv('data_sgd/test.tsv', header=None, sep="\t")
df2.head()

,0,1
0,__label__23,sounds good. can you find me a rental car ther...
1,__label__26,i need to check weather on the 5th of march.
2,__label__24,cool. schedule a visit.
3,__label__14,i would like to get a cab going there.
4,__label__1,would you like some tickets?


In [6]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 80#250
# This is fixed.
EMBEDDING_DIM = 100
HIDDEN_DIM = 64

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df1[1].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 4476 unique tokens.


In [7]:
X1 = tokenizer.texts_to_sequences(df1[1].values)
X1 = pad_sequences(X1, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X1.shape)

Shape of data tensor: (47022, 80)


In [8]:
Y1 = pd.get_dummies(df1[0]).values
print('Shape of label tensor:', Y1.shape)

Shape of label tensor: (47022, 21)


In [9]:
X2 = tokenizer.texts_to_sequences(df2[1].values)
X2 = pad_sequences(X2, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X2.shape)

Shape of data tensor: (9838, 80)


In [10]:
Y2 = pd.get_dummies(df2[0]).values
print('Shape of label tensor:', Y2.shape)

Shape of label tensor: (9838, 21)


In [11]:
X_train, X_test, Y_train, Y_test = X1,X2, Y1,Y2
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(47022, 80) (47022, 21)
(9838, 80) (9838, 21)


In [12]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [13]:
from keras.layers import GRU, RNN, Bidirectional

In [14]:
#model.add(Bidirectional(LSTM(64)))

#model = Sequential()
#model.add(Embedding(max_num_words, 100, input_length=max_length))
#model.add(Bidirectional(LSTM(units=128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
#model.add(SeqSelfAttention(attention_activation='sigmoid'))
#model.add(Flatten())
#model.add(Dense(1, activation='sigmoid'))

In [15]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X1.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(HIDDEN_DIM, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m]) # metrics=['accuracy']
print(model.summary())





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 80, 100)           5000000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 80, 100)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_1 (Dense)              (None, 21)                1365      
Total params: 5,043,605
Trainable params: 5,043,605
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 42319 samples, validate on 4703 samples
Epoch 1/5





42319/42319 [==============================] - 230s 5ms/step - loss: 1.0645 - acc: 0.7028 - f1_m: 0.6522 - precision_m: 0.8575 - recall_m: 0.5706 - val_loss: 0.4039 - val_acc: 0.8760 - val_f1_m: 0.8708 - val_precision_m: 0.9139 - val_recall_m: 0.8322
Epoch 2/5
42319/42319 [==============================] - 227s 5ms/step - loss: 0.3452 - acc: 0.8877 - f1_m: 0.8863 - precision_m: 0.9167 - recall_m: 0.8582 - val_loss: 0.3283 - val_acc: 0.8888 - val_f1_m: 0.8888 - val_precision_m: 0.9119 - val_recall_m: 0.8671
Epoch 3/5
42319/42319 [==============================] - 229s 5ms/step - loss: 0.2882 - acc: 0.8983 - f1_m: 0.8989 - precision_m: 0.9213 - recall_m: 0.8780 - val_loss: 0.3048 - val_acc: 0.8911 - val_f1_m: 0.8934 - val_precision_m: 0.9162 - val_recall_m: 0.8720
Epoch 4/5
42319/42319 [==============================] - 223s 5ms/

In [17]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

9838/9838 [==============================] - 5s 554us/step
Test set
  Loss: 0.280
  Accuracy: 0.902


In [18]:
#loss, accuracy, f1_score, precision, recall
print(accr)

[0.28013684569541264, 0.9015043707707229, 0.9058497637649647, 0.9235029212462512, 0.8894084163084338]


In [19]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(np.argmax(Y_test, axis=1), y_pred_bool, digits=4))

9838/9838 [==============================] - 6s 582us/step
              precision    recall  f1-score   support

           0     0.7887    0.8634    0.8243       549
           1     0.9162    0.9592    0.9372       319
           2     0.9344    0.9500    0.9421       180
           3     0.9210    0.9926    0.9554       270
           4     0.9356    0.9643    0.9497       196
           5     0.9045    0.8256    0.8632       367
           6     0.8688    0.5055    0.6391       275
           7     0.9471    0.8524    0.8972       210
           8     0.9543    0.9566    0.9555       415
           9     0.9250    0.9367    0.9308       158
          10     0.7596    0.8971    0.8226       243
          11     0.9290    0.9524    0.9405       357
          12     0.8054    0.8209    0.8131       363
          13     0.8472    0.9515    0.8963       577
          14     0.8816    0.7201    0.7927       393
          15     0.9858    0.9765    0.9811       213
          16     0.965